# 原生爬虫
* 明确目的
* 找到数据对应的网页
* 分析网页的结构，以及数据所在的标签位置

## 使用库
* Beautiful Soup 正则表达式
* Scrapy 爬虫框架

## 反爬虫
* ip --> 代理ip库

## 模拟服务器请求
获取到服务器返回给我们的HTML
用正则表达式提取我们要的数据(名字、人气) 

## 模块代码测试

In [4]:
from urllib import request
from io import BytesIO
import gzip

r = request.urlopen('https://www.douyu.com/g_LOL')
htmls = r.read()
buff = BytesIO(htmls)
f = gzip.GzipFile(fileobj=buff)
htmls = f.read().decode('utf-8')
# 这里是用gzip压缩过的代码，需要解码
htmls

'<!-- 分区详情 --><!-- 分区定制化 --><!doctype html><!--[if lte IE 7]><html class="ie ie7 acie"><![endif]--><!--[if IE 8]><html class="ie ie8"><![endif]--><!--[if IE 9]><html class="ie ie9"><![endif]--><html><head><meta charset="UTF-8"><title>英雄联盟直播_lol直播_英雄联盟直播视频解说攻略_斗鱼直播</title><meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1"><meta name="renderer" content="webkit"><meta name="keywords" content="英雄联盟,lol,英雄联盟直播,lol直播,英雄联盟直播间,英雄联盟直播地址,网游竞技直播,英雄联盟视频解说攻略,热门游戏直播,高清游戏直播,电子竞技直播,lol大神直播"><meta name="description" content="斗鱼英雄联盟直播专区拥有海量的英雄联盟lol直播内容,热门英雄联盟赛事直播解说和精彩lol英雄联盟直播内容全天候不间断,英雄联盟名玩大神主播芜湖大司马、东北大鹌鹑、洞主凯哥在这里带你各种操作秀翻天,一起嗨翻全场,全新lol英雄联盟直播尽在斗鱼直播。"><script>var serviceWorker=navigator.serviceWorker;serviceWorker&&(serviceWorker.getRegistrations?serviceWorker.getRegistrations().then(function(e){e.forEach(function(e){e.unregister()})}):serviceWorker.getRegistration&&serviceWorker.getRegistration().then(function(e){e&&e.unregister()}))</script><meta http-equiv="Content-Security-Policy" content="

In [5]:
# 正则表达式测试

In [17]:
import re
root_pattern = '<div class="DyListCover-info">[\s\S]*?</div>'
root_htmls = re.findall(root_pattern, htmls)
root_htmls = list(root_htmls)[1::2]
print(root_htmls)

['<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>71.1万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>主播油条</h2></div>', '<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>52.2万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>仙凡哥哥丶</h2></div>', '<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>71万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>南波儿大魔王丶</h2></div>', '<div class="DyListCover-info"><span class="DyListCover-hot is-template"><svg><use xlink:href="#icon-hot_8a57f0b"></use></svg>39.4万</span><h2 class="DyListCover-user is-template"><svg><use xlink:href="#icon-user_c95acf8"></use></svg>灵魂刀神</h2></

In [43]:
'''
'''
from urllib import request
from io import BytesIO
import gzip
import re

class Spider():
    url = 'https://www.douyu.com/g_LOL'
    
    # 注意里面原生html中为双引号
    # 正则表达式：匹配中间所有内容，非贪婪模式,自己体会，给需要匹配的内容增加括号成为组
    root_pattern = '<div class="DyListCover-info">([\s\S]*?)</div>'
    name_pattern1 = '</svg>([\s\S]*?)</h2>'
    name_pattern2 = '</svg>(.*)'
    number_pattern = '</svg>([\s\S]*?)</span>'
    
    def __fetch_content(self):
        r = request.urlopen(Spider.url)
        
        # 转码
        htmls = r.read()
        
        # 由于经过压缩，所以需要解压
        buff = BytesIO(htmls)
        f = gzip.GzipFile(fileobj=buff)
        htmls = f.read().decode('utf-8')
        return htmls
    
    def __analysis(self,htmls):
        root_htmls = re.findall(Spider.root_pattern, htmls)
        root_htmls = list(root_htmls)[1::2]
        anchors = []
        for html in root_htmls:
            name = re.findall(Spider.name_pattern1,html)
            
            # 注意这里正则表达式匹配以后返回的值类型是list
            name = re.findall(Spider.name_pattern2,name[0])
            number = re.findall(Spider.number_pattern,html)
            anchor = {'name':name,'number':number}
            anchors.append(anchor)
        return anchors
    
    def __refine(self,anchors):
        '''
        格式化匹配内容：去除无用字符
        string.strip() 去除前后空白字符串的内置函数
        '''
        l = lambda anchor:{
            'name':anchor['name'][0].strip(),
            'number':anchor['number'][0]
        }
        return map(l,anchors)
        
    def __sort(self,anchors):
        # key传入函数需要注意
        anchors = sorted(anchors,key=self.__sort_seed,reverse=True)
        return anchors
    
    def __sort_seed(self,anchor):
        r = re.findall('\d*',anchor['number'])
        number = float(r[0])
        if '万' in anchor['number']:
            number *= 10000
        return number
    
    def __show(self,anchors):
        for rank in range(0,len(anchors)):
            print('rank'+str(rank+1)
                 + ':'+anchors[rank]['name']
                 + ' '+anchors[rank]['number'])
    
    # 总控函数
    def go(self): 
        htmls = self.__fetch_content()
        anchors = self.__analysis(htmls)
        anchors = list(self.__refine(anchors))
        anchors = self.__sort(anchors)
        self.__show(anchors)

spider = Spider()
spider.go()

rank1:智勋勋勋勋 212.8万
rank2:主播油条 74.2万
rank3:仙凡哥哥丶 52万
rank4:疯狗炼金 38.9万
rank5:英雄联盟赛事 30万
rank6:中年人拳师七号 29.1万
rank7:抗寒使者 28.6万
rank8:暴走涵涵aa 24.5万
rank9:主播浩钦 21.5万
rank10:刀疤城丶你41朵小菊花 19.9万
rank11:古月泽宇Moonnnn 19.9万
rank12:诺夏辅助 19.7万
rank13:晴空空空空妹 18.5万
rank14:特工丶琪丨武汉加油 18.8万
rank15:琦琦7 18.2万
rank16:捕捉一只三吃 16.5万
rank17:自宇666 15.9万
rank18:苏轼丶Sush4 15.1万
rank19:斗鱼丶华仔 14万
rank20:小蝌蚪羊月 13.7万
rank21:广广l 13.5万
rank22:法神瑞兹 13.1万
rank23:雯神 12.9万
rank24:33超甜的 12.1万
rank25:解说十一 12万
rank26:斧头帮疾风奥拉夫 11.7万
rank27:是空格啊 11.4万
rank28:俞俊0829 11.2万
rank29:玩剑圣的毛毛虫 11.1万
rank30:熬夜的au 10.7万
rank31:隐藏路人中的小兵 10.7万
rank32:彭枪枪OVO 10.6万
rank33:小埃及热狗OB 10.3万
rank34:月野兔Kilo 9.5万
rank35:小诸葛Zz 9.4万
rank36:果然心动a 9.4万
rank37:店长盖伦 9.3万
rank38:唐人龙龟 9.2万
rank39:张烯烯 9.1万
rank40:啊三三三三三三三 8.9万
rank41:豪男xhn 8.8万
rank42:红茶bt 8.2万
rank43:吃货丶权爷 8万
rank44:猫咪老师ADKing 7.9万
rank45:狼神丶欧阳 7.9万
rank46:JDG丶imp 7.4万
rank47:一只小媚媚丶 6.9万
rank48:RNG上校OB皇族RYL不放弃 6.8万
rank49:卡萨鹏丶卡萨丁 6.6万
rank50:暴风工作室丶 6.5万
rank51:东灏盖伦 6.5万
rank52:只属于你de小可爱 6.4万
ran

爬取框架节点：
document.querySelector("#listAll > div.layout-Module-container.layout-Cover.ListContent > ul > li:nth-child(1) > div > a.DyListCover-wrap > div.DyListCover-content > div:nth-child(1)")
```html
<div class="DyListCover-info">
   <span class="DyListCover-hot"><svg class="DyListCover-hotIcon"><use xlink:href="#icon-hot_8a57f0b"></use></svg>39.3万</span>
    <h2 class="DyListCover-user"><svg class="DyListCover-userIcon"><use xlink:href="#icon-user_c95acf8"></use></svg>峡谷杀神酱油瓶</h2>
</div>
```